This is a starter notebook for the project, you'll have to import the libraries you'll need, you can find a list of the ones available in this workspace in the requirements.txt file in this workspace. 

In [1]:
# import os

# os.environ["OPENAI_API_KEY"] = "YOUR API KEY"
# os.environ["OPENAI_API_BASE"] = "https://openai.vocareum.com/v1"

# from langchain.llms import OpenAI

In [1]:
import os
from real_estate_db import RealEstateDBManager
from embedding_utils import get_embedder #, USE_OPENAI
from rag_pipeline import RAGEngine

In [2]:
DB_PATH = os.getenv("LANCEDB_PATH", "../../../../data/GenAI/05_project/lancedb_store")
json_path = "listings.json"

In [3]:
json_path = "listings.json"
manager = RealEstateDBManager(DB_PATH)
manager.create_table(force=True)

Dropped old table: real_estate_listing
Created new LanceDB table: real_estate_listing


In [4]:
embed_fn = get_embedder()

# Computes embeddings and stores in LanceDB
manager.ingest_listings(json_path, embed_fn)

Added 10 new listings


In [5]:
# optional: build index if supported
manager.create_vector_index(index_type="hnsw", metric="cosine")

Could not create index (maybe your LanceDB version lacks this API). LanceTable.create_index() got multiple values for argument 'metric'


In [6]:
# RAG engine
rag = RAGEngine(manager, embed_fn)

In [7]:
# Query
user_query = "Looking for a modern 3-bedroom with good schools, near public transit, under $1.5M"
result = rag.query(user_query, k=5)
print("Result:")
print(result)

LanceDB.search() not vailable of failed: name 'query' is not defined
Result:
{'answer': "Based on your requirements for a modern 3-bedroom home with good schools, near public transit, and under $1.5M, here are the top 3 recommended listings:\n\n1. Listing ID b90e501a0057fd472627d0a0631e55c4 (score=0.4014):\n   - Neighborhood: Oak Hill\n   - Price: $780,000\n   - Bedrooms: 4, Bathrooms: 3\n   - Size: 2,700 sqft\n   - Details: This stunning 4-bedroom, 3-bathroom home in Oak Hill offers a contemporary design, high-end finishes, and a family-friendly environment with excellent schools. The property is within your budget and provides an ideal living space for a family. While it has an extra bedroom compared to your preference, the additional space could be utilized as a home office or guest room.\n\n2. Listing ID 026664fddbf4585689d527cd7ccedcc7 (score=0.3949):\n   - Neighborhood: Sunnyvale\n   - Price: $450,000\n   - Bedrooms: 2, Bathrooms: 1\n   - Size: 1,100 sqft\n   - Details: This char